实验R2-R3：

In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import keras

from keras import layers,metrics
from keras.optimizers import RMSprop
from keras.layers import Dense, LSTM, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU, ELU
from keras.models import Sequential, load_model
from keras.utils import np_utils, multi_gpu_model

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder 
from sklearn.utils import shuffle as reset
from sklearn.model_selection import train_test_split,GridSearchCV,StratifiedShuffleSplit
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import log_loss,make_scorer

from matplotlib.colors import LogNorm
# import 
from matplotlib.pylab import plt
from copy import deepcopy
from datetime import datetime
from imblearn.over_sampling import RandomOverSampler #https://imbalanced-learn.org/stable/generated/imblearn.over_sampling.RandomOverSampler.html?highlight=randomoversampler
from frplayer import FilterResponseNormalization

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:
def train_test_split_DataFrame(data, test_size=0.2, considerTime=True, random_state=None):
    # ConsiderTime-------trainDF和testDF分割时是否考虑时间问题，即是否需要随机打乱。True:按照‘Dates’列进行降序排列,False：随机打乱样本的顺序，
    if considerTime:
        data=data.sort_values(by="Dates", ascending=True)
    else:
        data=reset(data, random_state=random_state)
    train=data[int(len(data)*test_size):].reset_index(drop=True)
    test=data[:int(len(data)*test_size)].reset_index(drop=True)
    return train, test

In [3]:
def parse_time(x):
    if '-' in x:
        DD=datetime.strptime(x,"%Y-%m-%d %H:%M:%S")#jjs
    else:
        DD=datetime.strptime(x,"%Y/%m/%d %H:%M")#zj    
    time=DD.hour#*60+DD.minute
    day=DD.day
    month=DD.month
    year=DD.year
    return time,day,month,year
def Dates2TDMY(x):
    if '-' in x:
        DD=datetime.strptime(x,"%Y-%m-%d %H:%M:%S")#jjs
    else:
        DD=datetime.strptime(x,"%Y/%m/%d %H:%M")#zj  
    time=DD.hour#*60+DD.minute
    day=DD.day
    month=DD.month
    year=DD.year
    #T_D_M_Y=str(time)+str(day)+str(month)+str(year)
    T_D_M_Y=str(time)+str(day)+str(month)
    return T_D_M_Y
def get_season(x):
    summer=0
    fall=0
    winter=0
    spring=0
    if (x in [5, 6, 7]):
        summer=1
    if (x in [8, 9, 10]):
        fall=1
    if (x in [11, 0, 1]):
        winter=1
    if (x in [2, 3, 4]):
        spring=1
    return summer, fall, winter, spring

In [4]:
def field2Vec(trainDF,testDF,fieldStr):
    fields=sorted(trainDF[fieldStr].unique())
    categories=sorted(trainDF["Category"].unique())
    C_counts=trainDF.groupby(["Category"]).size()
    F_C_counts=trainDF.groupby([fieldStr,"Category"]).size()
    F_counts=trainDF.groupby([fieldStr]).size()
    logodds={}
    logoddsPF={}
    MIN_CAT_COUNTS=2
    default_logodds=np.log(C_counts/len(trainDF))-np.log(1.0-C_counts/float(len(trainDF)))
    for f in fields:
        PA=F_counts[f]/float(len(trainDF))
        logoddsPF[f]=np.log(PA)-np.log(1.-PA)
        logodds[f]=deepcopy(default_logodds)
        for cat in F_C_counts[f].keys():
            if (F_C_counts[f][cat]>MIN_CAT_COUNTS) and F_C_counts[f][cat]<F_counts[f]:
                PA=F_C_counts[f][cat]/float(F_counts[f])
                logodds[f][categories.index(cat)]=np.log(PA)-np.log(1.0-PA)
        logodds[f]=pd.Series(logodds[f])
        logodds[f].index=range(len(categories))
    ########此部分代码，从逻辑上不应该出现在此处，但是为了编程的方便，放在了此处#########
    #fieldsTest=sorted(testDF[fieldStr].unique())
    #N_count=0
    #for f in fieldsTest:
        #if f not in fields:
            #logoddsPF[f]=-50.0  #np.log(0.)-np.log(1.)=-inf,便于计算，改为-99999.0
            #logodds[f]=deepcopy(default_logodds)
            #pa=1.0/float(len(categories))
            #logodds[f][range(len(categories))]=np.log(pa)-np.log(1.0-pa)
            #logodds[f]=pd.Series(logodds[f])
            #logodds[f].index=range(len(categories))
            #N_count=N_count+1
    #print(fieldStr+' N_count: '+str(N_count))
    ########此部分代码，从逻辑上不应该出现在此处，但是为了编程的方便，放在了此处#########
    #引进代码原作者的新思想
    if testDF.shape[0]>0: #如果testDF里有样本,......
        print('There are some new:'+fieldStr)
        new_fields=sorted(testDF[fieldStr].unique())
        new_F_counts=testDF.groupby(fieldStr).size()
        only_new=set(new_fields+fields)-set(fields)
        only_old=set(new_fields+fields)-set(new_fields)
        in_both=set(new_fields).intersection(fields)
        print('# only_new_fieldds:'+str(len(only_new)))
        for f in only_new:
            PA=new_F_counts[f]/float(len(testDF)+len(trainDF))
            logoddsPF[f]=np.log(PA)-np.log(1.-PA)
            logodds[f]=deepcopy(default_logodds)
            logodds[f].index=range(len(categories))
        for f in in_both:
            PA=(F_counts[f]+new_F_counts[f])/float(len(testDF)+len(trainDF))
            logoddsPF[f]=np.log(PA)-np.log(1.-PA)    
    return logodds,logoddsPF

In [5]:
def parse_data(df,logodds_A,logoddsPF_A,logodds_T,logoddsPF_T,needT_D_M_Y=False):
    feature_list=df.columns.tolist()
    if "Descript" in feature_list:
        feature_list.remove("Descript")
    if "Resolution" in feature_list:
        feature_list.remove("Resolution")
    if "Category" in feature_list:
        feature_list.remove("Category")
    if "Id" in feature_list:
        feature_list.remove("Id")

    cleanData=df[feature_list]
    cleanData.index=range(len(df))
    print("Creating address features")###Creating address features###
    address_features=cleanData["Address"].apply(lambda x: logodds_A[x])
    address_features.columns=["logodds_A"+str(x) for x in range(len(address_features.columns))]
    if needT_D_M_Y:
        print("Creating time T_D_M_Y features")###Creating time T_D_M_Y features###
        T_D_M_Y_features=cleanData["T_D_M_Y"].apply(lambda xx: logodds_T[xx])
        T_D_M_Y_features.columns=["logodds_T"+str(xx) for xx in range(len(T_D_M_Y_features.columns))]

    print("Parsing dates")            ###Creating address features###
    cleanData["Time"], cleanData["Day"], cleanData["Month"], cleanData["Year"]=zip(*cleanData["Dates"].apply(parse_time))
    #     dummy_ranks_DAY = pd.get_dummies(cleanData['DayOfWeek'], prefix='DAY')
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    #     cleanData["DayOfWeek"]=cleanData["DayOfWeek"].apply(lambda x: days.index(x)/float(len(days)))
    print("Creating one-hot variables")
    dummy_ranks_PD = pd.get_dummies(cleanData['PdDistrict'], prefix='PD')
    dummy_ranks_DAY = pd.get_dummies(cleanData["DayOfWeek"], prefix='DAY')
    cleanData["IsInterection"]=cleanData["Address"].apply(lambda x: 1 if "/" in x else 0)
    cleanData["logoddsPF_A"]=cleanData["Address"].apply(lambda x: logoddsPF_A[x])
    if needT_D_M_Y:
        cleanData["logoddsPF_T"]=cleanData["T_D_M_Y"].apply(lambda x: logoddsPF_T[x])
    print("droping processed columns")
    cleanData=cleanData.drop("PdDistrict",axis=1)
    cleanData=cleanData.drop("DayOfWeek",axis=1)
    cleanData=cleanData.drop("Address",axis=1)    
    cleanData=cleanData.drop("Dates",axis=1)
    if needT_D_M_Y:
        cleanData=cleanData.drop("T_D_M_Y",axis=1)
    feature_list=cleanData.columns.tolist()
    print("joining one-hot features")
    if needT_D_M_Y:
        features = cleanData[feature_list].join(dummy_ranks_PD.iloc[:,:]).join(dummy_ranks_DAY.iloc[:,:]).join(address_features.iloc[:,:]).join(T_D_M_Y_features.iloc[:,:])
    else:
        features = cleanData[feature_list].join(dummy_ranks_PD.iloc[:,:]).join(dummy_ranks_DAY.iloc[:,:]).join(address_features.iloc[:,:])
    print("creating new features")
    features["IsDup"]=pd.Series(features.duplicated()|features.duplicated(keep='last')).apply(int)
    features["Awake"]=features["Time"].apply(lambda x: 1 if (x==0 or (x>=8 and x<=23)) else 0)
    features["Summer"], features["Fall"], features["Winter"], features["Spring"]=zip(*features["Month"].apply(get_season))
    if "Category" in df.columns:
        labels = df["Category"].astype('category')
    else:
        labels=None
    return features,labels

In [6]:
def generator(X, Y, lookback, delay, min_index, max_index, shuffle=False, batch_size=128, step=6):
    if max_index is None:
        max_index = len(X) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(min_index + lookback, max_index, size=batch_size)#数值在[low, high)区间。
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)

        samples = np.zeros((len(rows), lookback // step, X.shape[-1]))
        targets = np.zeros((len(rows),Y.shape[1]))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = X[indices]
            targets[j] = Y[rows[j]+delay]
#         print('# row of Val: '+str(targets.shape[0]))###Tian
        yield samples, targets
    #Now here is the data generator that we will use. It yields a tuple (samples, targets) where samples is one batch of input data and targets is the corresponding array of target temperatures. It takes the following arguments:
        # •data: The original array of floating point data, which we just normalized in the code snippet above.
        # •lookback: How many timesteps back should our input data go.
        # •delay: How many timesteps in the future should our target be.
        # •min_index and max_index: Indices in the data array that delimit which timesteps to draw from. This is useful for keeping a segment of the data for validation and another one for testing.
        # •shuffle: Whether to shuffle our samples or draw them in chronological order.
        # •batch_size: The number of samples per batch.
        # •step: The period, in timesteps, at which we sample data. We will set it 6 in order to draw one data point every hour.

In [7]:
#Import data
ConsiderTime=True#False# True##trainDF和testDF分割时是否考虑时间问题，即是否需要随机打乱。True:按照‘Dates’列进行降序排列,False：随机打乱样本的顺序，
Rate_ALL=0.0 #0.0即不保留测试机
needOverSampler=False
needT_D_M_Y=True #False  使用_T_D_M_Y和周几
allDF=pd.read_csv("./train_addrCorrect.csv")
print('The shape of OrginalAllDF: '+str(allDF.shape))

xy_scaler=preprocessing.StandardScaler()
xy_scaler.fit(allDF[["X","Y"]])
allDF[["X","Y"]]=xy_scaler.transform(allDF[["X","Y"]])
allDF=allDF[abs(allDF["Y"])<100]
allDF.index=range(len(allDF))
print('The shape of AllDF after del wrong X and Y values: '+str(allDF.shape))

def listCat(x):
    return list(x)
allDF.drop_duplicates(inplace=True,subset=['Dates', 'DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y', 'Category'])
Train_duplicated=pd.pivot_table(allDF,index=['Dates','DayOfWeek','PdDistrict', 'Address', 'X', 'Y'], values='Category',aggfunc=[len,listCat])
print('The shape of AllDF after drop_duplicates: '+str(allDF.shape))
print(Train_duplicated.shape)

trainDF,testDF=train_test_split_DataFrame(allDF, test_size=Rate_ALL, considerTime=ConsiderTime, random_state=None)
print('Address_counts_allDF_trainDF_testDF: ' + str(len(allDF["Address"].unique())) + '_'+ str(len(trainDF["Address"].unique())) + '_' + str(len(testDF["Address"].unique())))

N_AllSample=allDF.shape[0]
N_AllTrain=trainDF.shape[0]
N_AllTest=testDF.shape[0]
N_CLASS=len(allDF["Category"].unique())
print('The # of AllDF, AllTrain, AllTest, is: '+str(N_AllSample)+','+str(N_AllTrain)+','+str(N_AllTest))
#################Now proceed as before#################
print('-----------LOGOODS: Address-------------')
logodds_A,logoddsPF_A=field2Vec(trainDF,testDF,"Address")
if needT_D_M_Y:
    trainDF["T_D_M_Y"]=trainDF["Dates"].apply(Dates2TDMY)
    trainDF["T_D_M_Y"]=trainDF["T_D_M_Y"]+trainDF["DayOfWeek"]
    if Rate_ALL>0:
        testDF[["X","Y"]]=xy_scaler.transform(testDF[["X","Y"]])
        testDF["T_D_M_Y"]=testDF["Dates"].apply(Dates2TDMY)
        testDF["T_D_M_Y"]=testDF["T_D_M_Y"]+testDF["DayOfWeek"]
    print('-----------LOGOODS: T_D_M_Y-------------')
    logodds_T,logoddsPF_T=field2Vec(trainDF,testDF,"T_D_M_Y")    
else:
    logodds_T=None
    logoddsPF_T=None
    
print('-----------LOGOODS: parse_data of Alltrain  -------------')
features, labels=parse_data(trainDF,logodds_A,logoddsPF_A,logodds_T,logoddsPF_T,needT_D_M_Y) 
if Rate_ALL>0:
    print('-----------LOGOODS: parse_data of Alltest  -------------')
    features_test, labels_test=parse_data(testDF,logodds_A,logoddsPF_A,logodds_T,logoddsPF_T,needT_D_M_Y)###########和训练集使用同样的时间和地点Logoodds值#####
    x_test=features_test.values
    y_test=labels_test.values
    y_test = keras.utils.to_categorical(LabelEncoder().fit_transform(np.array(y_test)), num_classes=N_CLASS)

print(features.columns.tolist())
print(len(features.columns))

collist=features.columns.tolist()
scaler = preprocessing.StandardScaler()
scaler.fit(features)
features[collist]=scaler.transform(features)
if Rate_ALL>0:
    features_test[collist]=scaler.transform(features_test)###########和训练集使用同样的scaler值#####
######################################################
#############################先进行过采样，然后再根据时间来排序##################################
if needOverSampler:
    print('------------RandomOverSampler--------------')
    ros = RandomOverSampler()
    featuresArrayOverSampler, labelsArrayOverSampler = ros.fit_resample(features.values,labels.values)#####过采样#####
    N_AllTrain_OverSampler=int(featuresArrayOverSampler.shape[0])
    print('Shape of OverSampler of AllTrain: '+str(featuresArrayOverSampler.shape))
else:
    featuresArrayOverSampler=features.values
    labelsArrayOverSampler=labels.values
    N_AllTrain_OverSampler=int(featuresArrayOverSampler.shape[0])
    print('------------Attention: we do not RandomOverSampler---------------')
if ConsiderTime:
    #####按照年（第6列）月（第5列）日（第4列）时（第3列）排序
    print('------------ConsiderTime:  Sorting--------------')
    time_temp=featuresArrayOverSampler[:,2]+np.dot(featuresArrayOverSampler[:,3],100)+np.dot(featuresArrayOverSampler[:,4],10000)+np.dot(featuresArrayOverSampler[:,5],1000000)
    features_label_time=np.column_stack((featuresArrayOverSampler,labelsArrayOverSampler))
    features_label_time=np.column_stack((features_label_time,time_temp))
    features_label_time =features_label_time[np.argsort(features_label_time[:,-1])]
    labelsArrayOverSampler=features_label_time[:,-2]
    featuresArrayOverSampler=features_label_time[:,0:featuresArrayOverSampler.shape[1]]
    del features_label_time
    #############################先进行过采样，然后再根据时间来排序----结束############################
if Rate_ALL>0:
    print('------------RandomOverSampler for AllTest--------------')
    ros = RandomOverSampler()
    featuresArray_test, labelsArray_test = ros.fit_resample(features_test.values,labels_test.values)#####过采样#####
    N_AllTest_OverSampler=int(featuresArray_test.shape[0])
    labelsArray_test = keras.utils.to_categorical(LabelEncoder().fit_transform(np.array(labelsArray_test)), num_classes=N_CLASS)
    print('Shape of OverSampler of AllTest: '+str(featuresArray_test.shape))

The shape of OrginalAllDF: (878049, 9)
The shape of AllDF after del wrong X and Y values: (877982, 9)
The shape of AllDF after drop_duplicates: (812529, 9)
(689038, 2)
Address_counts_allDF_trainDF_testDF: 23191_23191_0
The # of AllDF, AllTrain, AllTest, is: 812529,812529,0
-----------LOGOODS: Address-------------
-----------LOGOODS: T_D_M_Y-------------
-----------LOGOODS: parse_data of Alltrain  -------------
Creating address features
Creating time T_D_M_Y features
Parsing dates


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Creating one-hot variables
droping processed columns
joining one-hot features
creating new features
['X', 'Y', 'Time', 'Day', 'Month', 'Year', 'IsInterection', 'logoddsPF_A', 'logoddsPF_T', 'PD_BAYVIEW', 'PD_CENTRAL', 'PD_INGLESIDE', 'PD_MISSION', 'PD_NORTHERN', 'PD_PARK', 'PD_RICHMOND', 'PD_SOUTHERN', 'PD_TARAVAL', 'PD_TENDERLOIN', 'DAY_Friday', 'DAY_Monday', 'DAY_Saturday', 'DAY_Sunday', 'DAY_Thursday', 'DAY_Tuesday', 'DAY_Wednesday', 'logodds_A0', 'logodds_A1', 'logodds_A2', 'logodds_A3', 'logodds_A4', 'logodds_A5', 'logodds_A6', 'logodds_A7', 'logodds_A8', 'logodds_A9', 'logodds_A10', 'logodds_A11', 'logodds_A12', 'logodds_A13', 'logodds_A14', 'logodds_A15', 'logodds_A16', 'logodds_A17', 'logodds_A18', 'logodds_A19', 'logodds_A20', 'logodds_A21', 'logodds_A22', 'logodds_A23', 'logodds_A24', 'logodds_A25', 'logodds_A26', 'logodds_A27', 'logodds_A28', 'logodds_A29', 'logodds_A30', 'logodds_A31', 'logodds_A32', 'logodds_A33', 'logodds_A34', 'logodds_A35', 'logodds_A36', 'logodds_A37',

In [8]:
####TEST DNN
print('------------Building LSTM model--------------')
ShuffleInTraining=True
N_EPOCHS_0=2
N_EPOCHS=1
N_HN_1=128
N_HN=64
N_BATCH=64

Rate_Val=0.8


N_Val_OverSampler=int(np.around(N_AllTrain_OverSampler*Rate_Val))
N_Train_OverSampler=int(N_AllTrain_OverSampler-N_Val_OverSampler)
print('N_Train_OverSampler= '+str(N_Train_OverSampler))
N_CLASS=len(allDF["Category"].unique())
input_dim=featuresArrayOverSampler.shape[1]
output_dim=N_CLASS

N_Split=500
BlockSize=int(np.floor(N_Val_OverSampler/N_Split))
print('BlockSize is: '+str(BlockSize)) 
lookback=int(1.5*BlockSize)
delay=-1

RNNmodel = Sequential()
RNNmodel.add(LSTM(N_HN_1,dropout=0.5, recurrent_dropout=0.5,return_sequences=True,input_shape=(None,input_dim)))
RNNmodel.add(LSTM(N_HN,dropout=0.5, recurrent_dropout=0.5,return_sequences=True,))
RNNmodel.add(LSTM(N_HN,dropout=0.5, recurrent_dropout=0.5))
RNNmodel.add(Dense(output_dim))
RNNmodel.add(Activation('softmax'))
RNNmodel.compile(loss='categorical_crossentropy', optimizer=RMSprop(),metrics=['accuracy', metrics.top_k_categorical_accuracy])

labelsArrayOverSampler_1hot=keras.utils.to_categorical(LabelEncoder().fit_transform(np.array(labelsArrayOverSampler)), num_classes=N_CLASS)

------------Building LSTM model--------------
N_Train_OverSampler= 162506
BlockSize is: 1300


In [9]:
print('--------------------------generator AllTrain_set, Train_set and Val_set for LSTM---------------------------------')
train_generator=generator(featuresArrayOverSampler, labelsArrayOverSampler_1hot, lookback=lookback, delay=delay, min_index=0, max_index=N_Train_OverSampler, shuffle=ShuffleInTraining, batch_size=N_BATCH, step=1)
val_generator=generator(featuresArrayOverSampler, labelsArrayOverSampler_1hot, lookback=lookback, delay=delay, min_index=N_Train_OverSampler-lookback, max_index=N_Train_OverSampler+1, shuffle=False, batch_size=N_BATCH, step=1)
#数值在[min_index, max_index)区间。当delay=1时，就是用[min_index, max_index)区间的样本预测，第max_index个样本。当delay=2时，就是预测第max_index+1个
train_steps= (N_Train_OverSampler-lookback) // N_BATCH
val_steps =  1 #(N_Val-lookback) // N_BATCH
# test_steps =(N_AllTest - lookback) // N_BATCH


--------------------------generator AllTrain_set, Train_set and Val_set for LSTM---------------------------------


In [ ]:
print('---------------------------------------LSTM GO GO GO!!!!---------------------------------------------')
# history = RNNmodel.fit_generator(train_generator,steps_per_epoch=10,epochs=N_EPOCHS_0,verbose=1)
history = RNNmodel.fit_generator(train_generator,steps_per_epoch=train_steps,epochs=N_EPOCHS_0,verbose=1,validation_data=val_generator,validation_steps=val_steps)
print('------------LSTM train finished--------------------')
RNNmodel.save('jjs_model_0203LSTMV1.h5')
# RNNmodel.evaluate_generator(val_generator,steps=1, callbacks=None,verbose=1)

In [17]:
RNNmodel=load_model('jjs_model_0203LSTMV1.h5')

In [18]:
lookback=int(BlockSize)

In [19]:
print('-----------------Start the loop training!!---------------------')
# Scores_all=np.zeros([N_Split,4])
# ACC=[0.0,0.0]
# val_ACC=[0.0,0.0]
# val_TopACC=[0.0,0.0]
# for i_s in range(N_Split):
#     train_generator=generator(featuresArrayOverSampler, labelsArrayOverSampler_1hot, lookback=lookback, delay=delay, min_index=N_Train_OverSampler+i_s*BlockSize, max_index=N_Train_OverSampler+(i_s+1)*BlockSize+1, shuffle=ShuffleInTraining, batch_size=N_BATCH, step=1)
#     print('i_s='+str(i_s)+ ' in '+ str(N_Split) +';  max_index='+str(N_Train_OverSampler+(i_s+1)*BlockSize+1))
#     val_generator=generator(featuresArrayOverSampler, labelsArrayOverSampler_1hot, lookback=lookback, delay=delay, min_index=N_Train_OverSampler+(i_s+1)*BlockSize-lookback, max_index=N_Train_OverSampler+(i_s+1)*BlockSize+1,shuffle=False, batch_size=1, step=BlockSize)
#     #history = RNNmodel.fit_generator(train_generator,steps_per_epoch=1,epochs=N_EPOCHS,verbose=1,validation_data=val_generator,validation_steps=(N_Val_OverSampler - lookback) // N_BATCH)
#     history = RNNmodel.fit_generator(train_generator,steps_per_epoch=17,epochs=1,verbose=2,validation_data=val_generator,validation_steps=1)

#     ACC.extend(history.history['accuracy'])
#     val_ACC.extend(history.history['val_accuracy'])
#     val_TopACC.extend(history.history['top_k_categorical_accuracy'])
#     if i_s%100==0:
#         print(ACC)
#         print(val_ACC)
#         print(val_TopACC)

ACC=[0.0,0.0]
val_ACC=[0.0,0.0]
val_TopACC=[0.0,0.0]
for i_s in range(N_Split):
    train_generator=generator(featuresArrayOverSampler, labelsArrayOverSampler_1hot, lookback=lookback, delay=delay, min_index=N_Train_OverSampler+i_s*BlockSize, max_index=N_Train_OverSampler+(i_s+1)*BlockSize+1, shuffle=ShuffleInTraining, batch_size=N_BATCH, step=1)
    print('i_s='+str(i_s)+ ' in '+ str(N_Split) +';  max_index='+str(N_Train_OverSampler+(i_s+1)*BlockSize+1))
    val_generator=generator(featuresArrayOverSampler, labelsArrayOverSampler_1hot, lookback=lookback, delay=delay, min_index=N_Train_OverSampler+(i_s+1)*BlockSize-lookback, max_index=N_Train_OverSampler+(i_s+1)*BlockSize+1,shuffle=False, batch_size=1, step=BlockSize)
    #history = RNNmodel.fit_generator(train_generator,steps_per_epoch=1,epochs=N_EPOCHS,verbose=1,validation_data=val_generator,validation_steps=(N_Val_OverSampler - lookback) // N_BATCH)
    history = RNNmodel.fit_generator(train_generator,steps_per_epoch=17,epochs=1,verbose=1,validation_data=val_generator,validation_steps=1)
    ACC.extend(history.history['accuracy'])
    val_ACC.extend(history.history['val_accuracy'])
    val_TopACC.extend(history.history['top_k_categorical_accuracy'])#
    if i_s%49==0:
        print(ACC)
        print(val_ACC)
        print(val_TopACC)
RNNmodel.save('jjs_model_0203LSTMV2.h5')        
print(val_ACC)
NP_ACC=np.array(ACC)
print(np.sum(NP_ACC)/500)
NP_val_ACC=np.array(val_ACC)
print(np.sum(NP_val_ACC)/500)
NP_val_ACCTop5=np.array(val_TopACC)
print(np.sum(NP_val_ACCTop5)/500)
#     loss_i, acc_i, top5acc_i=RNNmodel.evaluate(x=x_val_i, y=y_val_i, batch_size=None, verbose=0, sample_weight=None, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False)
# print('i_s='+str(i_s)+' , '+str(np.array([[loss_i, acc_i, top5acc_i]])))
#     Scores_all[i_s,:]=np.array([[loss_i, acc_i, top5acc_i]])# np.mean(Scores_all, axis=0)    

-----------------Start the loop training!!---------------------
i_s=0 in 500;  max_index=163807
Epoch 1/1
17/17 [==============================] - 56s 3s/step - loss: 0.4498 - accuracy: 0.9393 - top_k_categorical_accuracy: 1.0000 - val_loss: 2.5109 - val_accuracy: 0.0000e+00 - val_top_k_categorical_accuracy: 1.0000
[0.0, 0.0, 0.9393382]
[0.0, 0.0, 0.0]
[0.0, 0.0, 1.0]
i_s=1 in 500;  max_index=165107
Epoch 1/1
17/17 [==============================] - 67s 4s/step - loss: 0.1233 - accuracy: 0.9917 - top_k_categorical_accuracy: 1.0000 - val_loss: 2.0893 - val_accuracy: 1.0000 - val_top_k_categorical_accuracy: 1.0000
i_s=2 in 500;  max_index=166407
Epoch 1/1
17/17 [==============================] - 78s 5s/step - loss: 2.2467 - accuracy: 0.4632 - top_k_categorical_accuracy: 0.7528 - val_loss: 3.9398 - val_accuracy: 0.0000e+00 - val_top_k_categorical_accuracy: 0.0000e+00
i_s=3 in 500;  max_index=167707
Epoch 1/1
17/17 [==============================] - 75s 4s/step - loss: 2.1141 - accuracy: 0

In [15]:
print(val_ACC)

NP_ACC=np.array(ACC)
print(np.sum(NP_ACC)/500)

NP_val_ACC=np.array(val_ACC)
print(np.sum(NP_val_ACC)/500)

NP_val_ACCTop5=np.array(val_TopACC)
print(np.sum(NP_val_ACCTop5)/500)

[0, 0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [ ]:
print('-----------------2222  Start the loop training!!---------------------')
# Scores_all=np.zeros([N_Split,4])
ACC2=[0,0]
val_ACC2=[0,0]
val_TopACC2=[0,0]
for i_s in range(N_Split):
    train_generator=generator(featuresArrayOverSampler, labelsArrayOverSampler_1hot, lookback=lookback, delay=delay, min_index=N_Train_OverSampler+i_s*BlockSize, max_index=N_Train_OverSampler+(i_s+1)*BlockSize+1, shuffle=ShuffleInTraining, batch_size=N_BATCH, step=1)
    print('i_s='+str(i_s)+ ' in '+ str(N_Split) +';  max_index='+str(N_Train_OverSampler+(i_s+1)*BlockSize+1))
    val_generator=generator(featuresArrayOverSampler, labelsArrayOverSampler_1hot, lookback=lookback, delay=delay, min_index=N_Train_OverSampler+(i_s+1)*BlockSize-lookback, max_index=N_Train_OverSampler+(i_s+1)*BlockSize+1,shuffle=False, batch_size=1, step=BlockSize)
    #history = RNNmodel.fit_generator(train_generator,steps_per_epoch=1,epochs=N_EPOCHS,verbose=1,validation_data=val_generator,validation_steps=(N_Val_OverSampler - lookback) // N_BATCH)
    history = RNNmodel.fit_generator(train_generator,steps_per_epoch=50,epochs=1,verbose=0,validation_data=val_generator,validation_steps=1)

    ACC2.extend(history.history['accuracy'])
    val_ACC2.extend(history.history['val_accuracy'])
    val_TopACC2.extend(history.history['top_k_categorical_accuracy'])
    if i_s%99==0:
        print(ACC2)
        print(val_ACC2)
        print(val_TopACC2)

-----------------2222  Start the loop training!!---------------------
i_s=0 in 500;  max_index=163807
[0, 0, 1.0]
[0, 0, 0.0]
[0, 0, 1.0]
i_s=1 in 500;  max_index=165107
i_s=2 in 500;  max_index=166407
i_s=3 in 500;  max_index=167707
i_s=4 in 500;  max_index=169007
i_s=5 in 500;  max_index=170307
i_s=6 in 500;  max_index=171607
i_s=7 in 500;  max_index=172907
i_s=8 in 500;  max_index=174207
i_s=9 in 500;  max_index=175507
i_s=10 in 500;  max_index=176807
i_s=11 in 500;  max_index=178107
i_s=12 in 500;  max_index=179407
i_s=13 in 500;  max_index=180707
i_s=14 in 500;  max_index=182007
i_s=15 in 500;  max_index=183307
i_s=16 in 500;  max_index=184607
i_s=17 in 500;  max_index=185907
i_s=18 in 500;  max_index=187207
i_s=19 in 500;  max_index=188507
i_s=20 in 500;  max_index=189807
i_s=21 in 500;  max_index=191107
i_s=22 in 500;  max_index=192407
i_s=23 in 500;  max_index=193707
i_s=24 in 500;  max_index=195007
i_s=25 in 500;  max_index=196307
i_s=26 in 500;  max_index=197607
i_s=27 in 500

In [26]:
a=5.0
b=3.0
div = a // b 
div
i_s=10000
i_s % 500==0

True

In [32]:
a=np.array([[1, 2, 3]])
print(a[0,1])
n=np.zeros([1,4])
n[0,1]=444
n

2


array([[  0., 444.,   0.,   0.]])